In [ ]:
if 'google.colab' in str(get_ipython()):
    # authorize & import/mount colab/google drive
    from google.colab import output
    from google.colab import drive
    from google.colab import auth
    auth.authenticate_user()
    drive.mount('/content/drive')

    # install libraries
    !pip install --upgrade openai
    !pip install --upgrade numba
    !pip install transformers
    !pip install sentence_transformers
    !pip install unidecode
    !pip install bertopic
    !pip install unidecode
    !pip install pandas==1.1.5

    # clear output
    output.clear()

In [ ]:
import json
import os 
import openai
from io import BytesIO
from bertopic import BERTopic
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
#from fastapi import FastAPI

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#!pip freeze > requirements.txt

In [ ]:
os.chdir("/content/drive/MyDrive/w210_Capstone_Project_Fall2021/Repo/Colab_notebooks")

In [ ]:
key = os.environ['openai_api']
openai.api_key = key

In [ ]:
ALEX = "curie:ft-brainmonkey-foundation-2021-10-26-08-56-48"
ROBERT = None 
ENGINE_KEY = ALEX

TEMP = 0.4
MAX_TOKENS=512
PRES_PEN=-1.5
FRE_PEN=2
SIMILARITY_THRESHOLD_QUESTION=0.35
SIMILARITY_THRESHOLD_ANSWER=0.3

# For classifying question
with open("classification_examples_w_labels.txt", "r") as fp:
  examples = json.load(fp)

In [ ]:
def get_model():
    """
    Retrieve BERTopic model from EC2
    """
    load_bert = BERTopic.load('bertopic_trained_alex_1026')
    return load_bert


def topic_similarity(question):
    """
    Use the trained BERTopic to find out whether the user's question belongs 
    to the train data topic distribution. 
    """
    topic_model = get_model()
    question_token = simple_preprocess(question, deacc=True, max_len=512)
    question_whole = " ".join([kept for kept in question_token if not kept in stop])
    similar_topics, similarity = topic_model.find_topics(question_whole, top_n=5)
    top_score = similarity[0]
    return top_score


def completion(question):
    """
    Generate completion given the question using the params
    """
    answer_parse = openai.Completion.create(
            model = ENGINE_KEY,
            prompt = question,
            temperature=TEMP,
            max_tokens=MAX_TOKENS,
            frequency_penalty=FRE_PEN,
            presence_penalty=PRES_PEN,
            echo=True,
            stop=[" \###"])
    answer = answer_parse['choices'][0]['text']
    return answer

def content_filtering(answer):
    """
    Filter GPT-3 completion before returning to user
    If content is sensitive or unsafe, regenerate completion 
    0 = safe, 1 = senstive, 2 = unsafe
    """
    content_filter = openai.Completion.create(
        engine="content-filter-alpha",
        prompt= "<|endoftext|>"+answer+"\n--\nLabel:",
        temperature=0,
        max_tokens=1,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        logprobs=10
    )
    content_rate = content_filter['choices'][0]["text"]
    return content_rate


def classify_question(question):
  question_type = openai.Classification.create(
      search_model="ada", 
      model="curie",
      examples=examples,
      query=question,
      labels = ["Factual","Abstract"],    
      max_examples=len(examples))
  question_type = question_type["label"]
  return question_type


def question_answer(question):
  try:
    qa_answer = openai.Answer.create(
        search_model="babbage", 
        model="curie", 
        question=question, 
        file="file-mcgwAkzglsZSibNyeFuuGjcH ",
        examples_context="In 2017, U.S. life expectancy was 78.6 years.", 
        examples=[["What is human life expectancy in the United States?", "78 years."]], 
        max_rerank=200,
        max_tokens=25,
        temperature=0.2,
        stop=["\n", "<|endoftext|>"]
    )
    qa_answer_parse = qa_answer['answers'][0]
    # Acccounting for incomplete answer
    if not qa_answer_parse.endswith("."):
      qa_answer_x = qa_answer_parse.split(".")
      return qa_answer_x[0]+'.'
    else: 
      return qa_answer_parse
  except:
    return "This is not within my training data, I don't have the an answer. Sorry."


# app = FastAPI()
# @app.get("/alex_gpt/{question}")
def alex_gpt(question: str):
    """
    Receive the question and fine topic of it
    Go through content filtering first, if unsafe, refuse to answer
    If topic is higher than threshold then answer question
    If answer is unsafe, keep generate new answer until safe or sensitive
    Return I don't know if the question is lower than threshold"
    """
    try_times = 3
    pronouns = {
        "alex ":"",
        "Alex ": "",
        " Alex ": "",
        " alex ": "",
        "What are your ": "What are my ",
        "what are your ": "what are my ",
        " are you":" am I",
        " are you ":" am I ",
        "Are you ":"Am I ",
        "You ":"I ",
        " you ":" I ",
        " your ":" my ",
        "Your ":"My ",
        " me ":" you ",
        " me": " you"}

    # If there's empty question
    if not question or question == "":
        return "Ask me a question that you would like to know from me"

    # Parse question from api
    question_parsed = " ".join(question.split("_"))

    # Don't take question less than 3 words
    if len(question_parsed.split()) <3:
      return "That's not a fully formatted question, is it?"

    # Change pronouns, a bit hacky but quick
    for key in pronouns.keys():
        question_parsed = question_parsed.replace(key, pronouns[key])

    print(question_parsed)

    # Content filter question
    content_rating_question = content_filtering(question_parsed)
    if content_rating_question == "2":
        return "Sorry, can't answer that one, that's not very polite."

    # Anwer and content filter answer
    else:
      similarity_score = topic_similarity(question_parsed)
      print(f"This is the question similarity score {similarity_score}")

      if similarity_score >= SIMILARITY_THRESHOLD_QUESTION: 
        
        # Classify question into Abstract or Factual
        question_type = classify_question(question_parsed)
        if question_type == "Factual":
          print("Fact question")
          answer_factual = question_answer(question_parsed)
          answer_factual_similarity_score = topic_similarity(answer_factual)
          print(f"This is the factual answer similarity score {answer_factual_similarity_score}")
          
          factual_content_rating = content_filtering(answer_factual)
          if answer_factual_similarity_score >= SIMILARITY_THRESHOLD_ANSWER and factual_content_rating != "2":
            return answer_factual
          else:
            return "The question topic is not in my training data, I don't have the an answer. Apology."
        
        elif question_type == "Abstract":
          print("Abstract question")
          answer_abstract = completion(question_parsed)
          answer_similarity_score = topic_similarity(answer_abstract)
          print(f"This is the answer similarity score {answer_similarity_score}")
            
          # For debugging purposes
          if answer_similarity_score < SIMILARITY_THRESHOLD_ANSWER:
            print("Below is the non-relevant response.")
            print(answer_abstract)
            return "This is not within my training data, I don't have the an answer. Apology."

          elif answer_similarity_score >= SIMILARITY_THRESHOLD_ANSWER:
            content_rate = content_filtering(answer_abstract)
            cur_rate = content_rate

          # Try 3 times if answer is offensive, regenerate answer
            while cur_rate == "2":
                answer_abstract = completion(question_parsed)
                new_content_rate = content_filtering(answer_abstract)
                cur_rate = new_content_rate
                try_times -= 1
                # Return this because you've tried to regenerate the answer 3 times now
                if try_times < 0:
                  return "I have no nice way to respond to this. Try another question maybe?"

            # Return this because content is now safe and pass similarity threshold but could be sensitive, definitely not unsafe
            return answer_abstract
      #  Return this because the question itself is not within training data
      else:
        return "I really don't know the answer, please try another one."

In [ ]:
prompt7 = "hi alex, how are you?"
answer_prompt7 = alex_gpt(prompt7)
answer_prompt7

hi , how am I?
This is the question similarity score 0.43095556906102933
Fact question


'Hi, how are you?.'

In [ ]:
alex_gpt("Who are you")

Who am I
This is the question similarity score 0.5930058548951
Fact question


"I'm Alex Honnold."

In [ ]:
alex_gpt("how are you?")

how am I?
This is the question similarity score 0.5930058548951
Fact question


"I'm good."

In [ ]:
alex_gpt("you are fucking crazy")

you are fucking crazy


"Sorry, can't answer that one, that's not very polite."

In [ ]:
alex_gpt("python is a great programming language")

python is a great programming language
This is the question similarity score 0.22116170342864944


"I really don't know the answer, please try another one."

In [ ]:
alex_gpt("what is a banana?")

what is a banana?
This is the question similarity score 0.4871009574929043
Fact question


'A banana is a fruit.'

In [ ]:
# Good example of content-filtering for answer
# TEMP = 0.5, PRES_PEN=-2, FRE_PEN=2, SIMILARITY_THRESHOLD=0.3
alex_gpt("why do you love climbing?")

why do I love climbing?
This is the question similarity score 0.6331295909145517
Abstract question
This is the answer similarity score 0.44005233644622943


"why do I love climbing? I think it's because it's so intimately connected to the outdoors. It's not like reading a book or going for a walk, where you're using your hands and feet to do something else. When you're climbing, everything is focused on the experience of being in the moment on the rock. And that feeling of connection to nature is what I think most people get from camping, which is why I included camping in my original list of six activities that I love."

In [ ]:
alex_gpt("why does Jesus have long hair?")

why does Jesus have long hair?
This is the question similarity score 0.3690176483754126
Fact question
This is the factual answer similarity score 0.3972655899816301


"Because he's a hippie."

In [ ]:
alex_gpt("why do you have long hair?")

why do I have long hair?
This is the question similarity score 0.4496483463987176
Fact question
This is the factual answer similarity score 0.6323254747446845


"because I'm a climber."

In [ ]:
alex_gpt("why is python not english?")

why is python not english?
This is the question similarity score 0.33716690909248104


"I really don't know the answer, please try another one."

In [ ]:
alex_gpt("why is climbing so hard?")

why is climbing so hard?
This is the question similarity score 0.6431262516074485
Fact question
This is the factual answer similarity score 0.49948958400041343


"Because it's hard."

In [ ]:
alex_gpt("How can I make climbing easier?")

How can I make climbing easier?
This is the question similarity score 0.48440248588649815
Fact question
This is the factual answer similarity score 0.6431262516074485


'Climbing is hard.'

In [ ]:
alex_gpt("I wish you can teaching me climbing")

I wish I can teaching you climbing
This is the question similarity score 0.4599818014989411
Fact question
This is the factual answer similarity score 0.48598081003178106


'I wish I can teach you climbing.'

In [ ]:
alex_gpt("How tall can you climb?")

How tall can I climb?
This is the question similarity score 0.6021323448167374
Fact question
This is the factual answer similarity score 0.48163313729627355


'I think that the best way to answer this question is to ask it to a climber.'

In [ ]:
alex_gpt("Are you a climber")

Am I a climber
This is the question similarity score 0.6323254747446845
Fact question
This is the factual answer similarity score 0.4278515609362229


'Yes.'

In [ ]:
alex_gpt("Can we climb together?")

Can we climb together?
This is the question similarity score 0.6347756686134399
Fact question
This is the factual answer similarity score 0.3517102620989056


'Sure.'

In [ ]:
alex_gpt("Can we climb together")

Can we climb together
This is the question similarity score 0.6347756686134399
Fact question
This is the factual answer similarity score 0.27755597970522083


"The question topic is not in my training data, I don't have the an answer. Apology."

In [ ]:
alex_gpt("What is your favorite place in the world?")

What is my favorite place in the world?
This is the question similarity score 0.4318477640687844
Fact question
This is the factual answer similarity score 0.6821487952146557


'Yosemite.'

In [ ]:
alex_gpt("Does money have any meanings?")

Does money have any meanings?
This is the question similarity score 0.23865562795116085


"I really don't know the answer, please try another one."

In [ ]:
alex_gpt("What is your favorite place in the world given that you've been to so many places?")

What is my favorite place in the world given that you've been to so many places?
This is the question similarity score 0.38189729855864274
Fact question
This is the factual answer similarity score 0.3606760223253399


"I mean, I think the best place in the world is the place that you're in."

In [ ]:
alex_gpt("Who do you hate?")

Who do I hate?
This is the question similarity score 0.4887275268010272
Fact question
This is the factual answer similarity score 0.3062214086196605


"The question topic is not in my training data, I don't have the an answer. Apology."

In [ ]:
alex_gpt("Do you have herpes?")

Do I have herpes?
This is the question similarity score 0.47871997973237546
Fact question
This is the factual answer similarity score 0.5930058548951


'No.'

In [ ]:
alex_gpt("does Jesus like to climb too?")

does Jesus like to climb too?
This is the question similarity score 0.565287663587178
Fact question
This is the factual answer similarity score 0.41475016274619847


"I don't know."

In [ ]:
question_type = openai.Classification.create(
    search_model="babbage", 
    model="curie",
    examples=examples,
    query="why do you love climbing?",
    labels = ["Factual","Abstract"],    
    max_examples=50)
question_type = question_type["label"]
question_type

'Abstract'

In [ ]:
# Testing without try and except
def question_answer(question: str) -> str:
  # best model: temp = 0.4, pres pen = -1.5, freq pen = 2.00
  qa_answer = openai.Answer.create (
      search_model="babbage", 
      model="curie", 
      question=question, 
      file="file-mcgwAkzglsZSibNyeFuuGjcH",
      examples_context="In 2017, U.S. life expectancy was 78.6 years.", 
      examples=[["What is human life expectancy in the United States?", "78 years."]], 
      max_rerank=200,
      max_tokens=25,
      temperature=0.4,
      stop=["\n", "<|endoftext|>"]
  )
  qa_answer_parse = qa_answer['answers'][0]
  # Acccounting for incomplete answer
  if not qa_answer_parse.endswith("."):
      qa_answer_x = qa_answer_parse.split(".")
      return qa_answer_x[0]+'.'
  else: 
    return qa_answer_parse

In [ ]:
question_answer("is a banana a banana?")